Data sources from https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
Excercise in producing a decision tree to predict forest cover of parcels of land based on the 50+ tributes.
Based on the case study in Advnaced Analytics using Spark
Using Almond Shell kernel for Jupyter notebook

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3`
import org.apache.spark.sql._

import $ivy.$                                  

import org.apache.spark.sql._

In [2]:
import org.apache.log4j.{Logger,Level}

import org.apache.log4j.{Logger,Level}

In [3]:
Logger.getLogger("org").setLevel(Level.ERROR)

Create the sql sessions 

In [4]:
val spark = {
            org.apache.spark.sql.NotebookSparkSession.builder()
            .master("local[*]")
            .getOrCreate()
            }

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@32e73e54

In [5]:
val data = spark.read
            .option("header","false")
            .option("inferSchema","true")
            .csv("../covtype/")

csv at cmd4.sc:4

1 / 1

csv at cmd4.sc:4

4 / 4

data: DataFrame = [_c0: string, _c1: string ... 53 more fields]

In [6]:
data.count()

count at cmd5.sc:1

4 / 4

count at cmd5.sc:1

1 / 1

res5: Long = 581259L

In [7]:
data.head()

head at cmd6.sc:1

1 / 1

res6: Row = [2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5]

In [8]:
data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)
 |-- _c15: integer (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: integer (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)
 |-- _c21: integer (nullable = true)
 |-- _c22: integer (nullable = true)
 |-- _c23: integer (nullable = true)
 |-- _c24: integer (nullable = true)
 |-- _c25: integer (nullable = true)
 |-- _c26: integer (nullable = true)
 |-- _c27

Looks like the schema isn't what was expected. _c0..2 should be integers

In [7]:
import org.apache.spark.sql.types._

import org.apache.spark.sql.types._

Have to update the scheme. Only way seems to be to create new DF every time.

In [8]:
val data2=data.withColumn("_c0",data("_c0").cast(IntegerType))
val data3=data2.withColumn("_c1",data2("_c1").cast(IntegerType))
val data4=data3.withColumn("_c2",data3("_c2").cast(IntegerType))

data2: DataFrame = [_c0: int, _c1: string ... 53 more fields]
data3: DataFrame = [_c0: int, _c1: int ... 53 more fields]
data4: DataFrame = [_c0: int, _c1: int ... 53 more fields]

In [9]:
data4.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)
 |-- _c15: integer (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: integer (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)
 |-- _c21: integer (nullable = true)
 |-- _c22: integer (nullable = true)
 |-- _c23: integer (nullable = true)
 |-- _c24: integer (nullable = true)
 |-- _c25: integer (nullable = true)
 |-- _c26: integer (nullable = true)
 |-- _

Above approach worked by duplicating the dataframe everytime seem thereis potential for memory overflow. Try doing it when adding titles.

Create header titles for the columns. 

In [10]:
val colNames = Seq(
"Elevation", "Aspect", "Slope",
"Horizontal_Distance_To_Hydrology", "Vertical_Distance_To_Hydrology",
"Horizontal_Distance_To_Roadways",
"Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm",
"Horizontal_Distance_To_Fire_Points"
) ++ (
(0 until 4).map(i => s"Wilderness_Area_$i")
) ++ (
(0 until 40).map(i => s"Soil_Type_$i")
) ++ Seq("Cover_Type")

colNames: Seq[String] = List(
  "Elevation",
  "Aspect",
  "Slope",
  "Horizontal_Distance_To_Hydrology",
  "Vertical_Distance_To_Hydrology",
  "Horizontal_Distance_To_Roadways",
  "Hillshade_9am",
  "Hillshade_Noon",
  "Hillshade_3pm",
  "Horizontal_Distance_To_Fire_Points",
  "Wilderness_Area_0",
  "Wilderness_Area_1",
  "Wilderness_Area_2",
  "Wilderness_Area_3",
  "Soil_Type_0",
  "Soil_Type_1",
  "Soil_Type_2",
  "Soil_Type_3",
  "Soil_Type_4",
  "Soil_Type_5",
  "Soil_Type_6",
  "Soil_Type_7",
  "Soil_Type_8",
  "Soil_Type_9",
  "Soil_Type_10",
  "Soil_Type_11",
  "Soil_Type_12",
  "Soil_Type_13",
  "Soil_Type_14",
  "Soil_Type_15",
  "Soil_Type_16",
  "Soil_Type_17",
  "Soil_Type_18",
  "Soil_Type_19",
  "Soil_Type_20",
  "Soil_Type_21",
  "Soil_Type_22",
  "Soil_Type_23",
...

In [11]:
colNames.getClass

res10: Class[T] = class scala.collection.immutable.$colon$colon

implicits contains the function $ which returns a column object. Needed to access the data type in order to cast it to integer. $("string"): column   ... pretty 'hacky' syntax.

In [12]:
import spark.implicits._

import spark.implicits._

In [13]:
val dataClean = data.toDF(colNames:_*).
    withColumn("Elevation",$"Elevation".cast(IntegerType))
    .withColumn("Aspect",$"Aspect".cast(IntegerType))
    .withColumn("Slope",$"Slope".cast(IntegerType))
    .withColumn("Cover_Type",$"Cover_Type".cast(DoubleType))


dataClean: DataFrame = [Elevation: int, Aspect: int ... 53 more fields]

In [14]:
dataClean.show() // Using the spark.sql API

show at cmd13.sc:1

1 / 1

+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+----------------------------------+-----------------+-----------------+-----------------+-----------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+----------+
|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|Hillshade_Noon|Hillshade_3pm|Horizontal_Distance_To_Fire_Points|Wilderness

|     2617|    45|    9|                             240|                            56|                            666|          223|           221|          133|                              6244|                1|                0|                0|                0|          0|          0|          0|          0|          0|          0|          0|          0|          0|          0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           1|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|       5.0|
|     2612|    59|   10|                             247|                            11|                            636|          228|           219|          124|                              6230|          

|     2504|     0|    4|                              95|                             5|                            691|          214|           232|          156|                              5572|                1|                0|                0|                0|          0|          0|          0|          0|          0|          0|          0|          0|          0|          0|           0|           0|           0|           0|           0|           0|           0|           1|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|       5.0|
|     2503|    38|    5|                              85|                            10|                            741|          220|           228|          144|                              5555|          

Randomly split the data into train and test using 10% for testing (will need to further split the training data if CV performed

In [15]:
val Array(trainData, testData) = dataClean.randomSplit(Array(0.9,0.1))
trainData.cache()
testData.cache()

trainData: Dataset[Row] = [Elevation: int, Aspect: int ... 53 more fields]
testData: Dataset[Row] = [Elevation: int, Aspect: int ... 53 more fields]
res14_1: Dataset[Row] = [Elevation: int, Aspect: int ... 53 more fields]
res14_2: Dataset[Row] = [Elevation: int, Aspect: int ... 53 more fields]

Vectorise the features. This is required by all SparkML alogorithms.

Took some messing around to find the right repo for ml lib. Eventually found a reference on the Mvnrepository site to find the mllib version that goes with scala 2.12.

In [16]:
import $ivy.`org.apache.spark::spark-mllib:2.4.0`

import $ivy.$                                    

In [17]:
import org.apache.spark.ml.feature.VectorAssembler

import org.apache.spark.ml.feature.VectorAssembler

In [18]:
val inputCols = trainData.columns.filter(_!="Cover_Type")
val assembler = new org.apache.spark.ml.feature.VectorAssembler()
                    .setInputCols(inputCols)
                    .setOutputCol("featureVector")

inputCols: Array[String] = Array(
  "Elevation",
  "Aspect",
  "Slope",
  "Horizontal_Distance_To_Hydrology",
  "Vertical_Distance_To_Hydrology",
  "Horizontal_Distance_To_Roadways",
  "Hillshade_9am",
  "Hillshade_Noon",
  "Hillshade_3pm",
  "Horizontal_Distance_To_Fire_Points",
  "Wilderness_Area_0",
  "Wilderness_Area_1",
  "Wilderness_Area_2",
  "Wilderness_Area_3",
  "Soil_Type_0",
  "Soil_Type_1",
  "Soil_Type_2",
  "Soil_Type_3",
  "Soil_Type_4",
  "Soil_Type_5",
  "Soil_Type_6",
  "Soil_Type_7",
  "Soil_Type_8",
  "Soil_Type_9",
  "Soil_Type_10",
  "Soil_Type_11",
  "Soil_Type_12",
  "Soil_Type_13",
  "Soil_Type_14",
  "Soil_Type_15",
  "Soil_Type_16",
  "Soil_Type_17",
  "Soil_Type_18",
  "Soil_Type_19",
  "Soil_Type_20",
  "Soil_Type_21",
  "Soil_Type_22",
  "Soil_Type_23",
...
assembler: VectorAssembler = vecAssembler_c8132b1f7834

Since spark 2.4 vector assembler extends HadHandleInvalid which requires you to specify how nulls will be handled. It take a single parameter ("skip") to skip when encountered. If not used then the dataset must be cleaned up and the missing data imputed. 

See stackoverflow
https://stackoverflow.com/questions/41362295/sparkexception-values-to-assemble-cannot-be-null


TODO: Investigate the source of nulls

In [19]:
val assembledTrainData = assembler.setHandleInvalid("skip").transform(trainData)

assembledTrainData: DataFrame = [Elevation: int, Aspect: int ... 54 more fields]

In [20]:
assembledTrainData.select("featureVector").show(truncate=false)

show at cmd19.sc:1

1 / 1

+-----------------------------------------------------------------------------------------------------+
|featureVector                                                                                        |
+-----------------------------------------------------------------------------------------------------+
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1863.0,37.0,17.0,120.0,18.0,90.0,217.0,202.0,115.0,769.0,1.0,1.0])  |
|(54,[0,1,2,5,6,7,8,9,13,18],[1874.0,18.0,14.0,90.0,208.0,209.0,135.0,793.0,1.0,1.0])                 |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1879.0,28.0,19.0,30.0,12.0,95.0,209.0,196.0,117.0,778.0,1.0,1.0])   |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1888.0,33.0,22.0,150.0,46.0,108.0,209.0,185.0,103.0,735.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,14],[1889.0,28.0,22.0,150.0,23.0,120.0,205.0,185.0,108.0,759.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1889.0,353.0,30.0,95.0,39.0,67.0,153.0,172.0,146.0,600.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1896.0,337.0,12.0,30.0,6.0,175

In [21]:
assembledTrainData.getClass

res20: Class[T] = class org.apache.spark.sql.Dataset

In [22]:
trainData.getClass

res21: Class[T] = class org.apache.spark.sql.Dataset

Note on above. The transform fucntion converts a dataframe to a dataframe but re-organises it.

Now to create the DecisionTreem classifier for predicting the group cover type (a categorical valiable ranging 1..7)

In [23]:
import org.apache.spark.ml.classification.DecisionTreeClassifier
import scala.util.Random

import org.apache.spark.ml.classification.DecisionTreeClassifier

import scala.util.Random

In [24]:
val classifier = new org.apache.spark.ml.classification.DecisionTreeClassifier()
                        .setSeed(Random.nextLong())
                        .setLabelCol("Cover_Type")
                        .setFeaturesCol("featureVector")
                        .setPredictionCol("prediction")


classifier: DecisionTreeClassifier = dtc_d40f44a481da

In [25]:
val model = classifier.fit(assembledTrainData)
//print(model.toDebugString)

take at Classifier.scala:111

4 / 4

take at Classifier.scala:111

1 / 1

take at DecisionTreeMetadata.scala:112

...

mapPartitions at RandomForest.scala:538

4 / 4

collectAsMap at RandomForest.scala:557

4 / 4

model: org.apache.spark.ml.classification.DecisionTreeClassificationModel = DecisionTreeClassificationModel (uid=dtc_d40f44a481da) of depth 5 with 51 nodes

In [26]:
print(model.toDebugString)

DecisionTreeClassificationModel (uid=dtc_d40f44a481da) of depth 5 with 51 nodes
  If (feature 0 <= 3034.5)
   If (feature 0 <= 2570.5)
    If (feature 10 <= 0.5)
     If (feature 0 <= 2461.5)
      If (feature 3 <= 15.0)
       Predict: 4.0
      Else (feature 3 > 15.0)
       Predict: 3.0
     Else (feature 0 > 2461.5)
      If (feature 17 <= 0.5)
       Predict: 2.0
      Else (feature 17 > 0.5)
       Predict: 3.0
    Else (feature 10 > 0.5)
     If (feature 9 <= 5421.0)
      Predict: 2.0
     Else (feature 9 > 5421.0)
      If (feature 31 <= 0.5)
       Predict: 2.0
      Else (feature 31 > 0.5)
       Predict: 5.0
   Else (feature 0 > 2570.5)
    If (feature 15 <= 0.5)
     If (feature 0 <= 2945.5)
      If (feature 17 <= 0.5)
       Predict: 2.0
      Else (feature 17 > 0.5)
       Predict: 3.0
     Else (feature 0 > 2945.5)
      Predict: 2.0
    Else (feature 15 > 0.5)
     If (feature 9 <= 1429.5)
      If (feature 7 <= 214.5)
       Predict: 2.0
      Else (feature 7 > 214.5

Cool! Above cell appears to show a recursive definition (and almost self explaining) of the decision tree model.
So now the model has been fit (estimator(ed) in spark speak) we can use the model to make predictions on the test data. 
Note that the default settings have been used on the above model so we don't expect this out of the box model to do very well.

Without getting under the hood I assume that the decison tree optimises the gini index for the tree. TODO find out.

DecisiontTreeClassifiers have an 'importance' dunction to assess the importance of each feature. Not obvious what this is but assume its some sort of sensitivity analysis.??

In [27]:
model.featureImportances.toArray.zip(inputCols).sorted.reverse.foreach(println)

(0.818216881787316,Elevation)
(0.03629006981274707,Wilderness_Area_0)
(0.02953263679555114,Soil_Type_3)
(0.028748729016733452,Horizontal_Distance_To_Hydrology)
(0.02779956715137853,Hillshade_Noon)
(0.02244096460214384,Soil_Type_1)
(0.01759182656196295,Soil_Type_31)
(0.011810373439182328,Wilderness_Area_2)
(0.0056349601227200915,Horizontal_Distance_To_Roadways)
(0.0016592025090799497,Horizontal_Distance_To_Fire_Points)
(2.7478820118465513E-4,Soil_Type_17)
(0.0,Wilderness_Area_3)
(0.0,Wilderness_Area_1)
(0.0,Vertical_Distance_To_Hydrology)
(0.0,Soil_Type_9)
(0.0,Soil_Type_8)
(0.0,Soil_Type_7)
(0.0,Soil_Type_6)
(0.0,Soil_Type_5)
(0.0,Soil_Type_4)
(0.0,Soil_Type_39)
(0.0,Soil_Type_38)
(0.0,Soil_Type_37)
(0.0,Soil_Type_36)
(0.0,Soil_Type_35)
(0.0,Soil_Type_34)
(0.0,Soil_Type_33)
(0.0,Soil_Type_32)
(0.0,Soil_Type_30)
(0.0,Soil_Type_29)
(0.0,Soil_Type_28)
(0.0,Soil_Type_27)
(0.0,Soil_Type_26)
(0.0,Soil_Type_25)
(0.0,Soil_Type_24)
(0.0,Soil_Type_23)
(0.0,Soil_Type_22)
(0.0,Soil_Type_21)
(0.0,S

So the above model has the elevation as by far and away the most important feature. However we have not assessed the performance of modey yet so this should be treated cautiously until this is done. 

Note that the decisionTreeClassificationModel is a transformed ( in spark speak this is just a function that transform a dataframe into a dataframe

In [28]:
val preds = model.transform(assembledTrainData)

preds.select("Cover_Type","prediction","probability").show(truncate=false)

show at cmd27.sc:3

1 / 1

+----------+----------+-------------------------------------------------------------------------------------------------+
|Cover_Type|prediction|probability                                                                                      |
+----------+----------+-------------------------------------------------------------------------------------------------+
|6.0       |3.0       |[0.0,0.0,0.038450094224220584,0.6285912652494462,0.04830231097298906,0.0,0.28465632955334413,0.0]|
|6.0       |4.0       |[0.0,0.0,0.04849000425350915,0.2871118672905147,0.4147171416418545,0.0,0.24968098681412165,0.0]  |
|6.0       |3.0       |[0.0,0.0,0.038450094224220584,0.6285912652494462,0.04830231097298906,0.0,0.28465632955334413,0.0]|
|6.0       |3.0       |[0.0,0.0,0.038450094224220584,0.6285912652494462,0.04830231097298906,0.0,0.28465632955334413,0.0]|
|6.0       |3.0       |[0.0,0.0,0.038450094224220584,0.6285912652494462,0.04830231097298906,0.0,0.28465632955334413,0.0]|
|6.0       |3.0       |[

preds: DataFrame = [Elevation: int, Aspect: int ... 57 more fields]

In [30]:
preds.columns.foreach(println)

class java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java.lang.Stringclass java

In [31]:
val results = preds.select("Cover_Type","prediction")

results: DataFrame = [Cover_Type: double, prediction: double]

In [32]:
results.selectExpr("Cover_Type==prediction as Matched","Cover_Type")
    .groupBy("Cover_Type").pivot("Matched",Seq("true","false")).count().show()

show at cmd31.sc:2

4 / 4

show at cmd31.sc:2

200 / 200

show at cmd31.sc:2

...

show at cmd31.sc:2

100 / 100

show at cmd31.sc:2

75 / 75

+----------+------+-----+
|Cover_Type|  true|false|
+----------+------+-----+
|       7.0|  6455|12003|
|       1.0|125719|64790|
|       4.0|   975| 1476|
|       3.0| 26109| 6214|
|       2.0|205689|49310|
|       6.0|  null|15591|
|       5.0|    75| 8487|
+----------+------+-----+



In [33]:
val predPivot = results.selectExpr("Cover_Type","prediction")
    .groupBy("Cover_Type")
    .pivot("prediction",(1 to 7))
    .count()
    .na.fill(0).sort("Cover_Type")

predPivot: Dataset[Row] = [Cover_Type: double, 1: bigint ... 6 more fields]

In [34]:
predPivot.show()

show at cmd33.sc:1

4 / 4

show at cmd33.sc:1

200 / 200

show at cmd33.sc:1

200 / 200

+----------+------+------+-----+---+---+---+----+
|Cover_Type|     1|     2|    3|  4|  5|  6|   7|
+----------+------+------+-----+---+---+---+----+
|       1.0|125719| 63152|  163|  0|  0|  0|1475|
|       2.0| 44736|205689| 4207|114| 64|  0| 189|
|       3.0|     0|  5539|26109|675|  0|  0|   0|
|       4.0|     0|    15| 1461|975|  0|  0|   0|
|       5.0|     0|  7857|  630|  0| 75|  0|   0|
|       6.0|     0|  5756| 9248|587|  0|  0|   0|
|       7.0| 11780|   171|   52|  0|  0|  0|6455|
+----------+------+------+-----+---+---+---+----+



TODO: Add row total

In [35]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

In [36]:
val evaluator = new org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator()
                .setLabelCol("Cover_Type")
                .setPredictionCol("prediction")

evaluator: MulticlassClassificationEvaluator = mcEval_150f2cdd1759

In [37]:
evaluator.setMetricName("accuracy").evaluate(preds)

map at MulticlassMetrics.scala:45

4 / 4

collectAsMap at MulticlassMetrics.scala:47

4 / 4

countByValue at MulticlassMetrics.scala:42

4 / 4

countByValue at MulticlassMetrics.scala:42

4 / 4

res36: Double = 0.6980816342922931

As expected accuracy not great. Also, looking at the prediction pivot table shows that there is class imbalance that might be affecting the utility of accuracy and f1 might be a better score.


In [38]:
evaluator.setMetricName("f1").evaluate(preds)

countByValue at MulticlassMetrics.scala:42

4 / 4

countByValue at MulticlassMetrics.scala:42

4 / 4

map at MulticlassMetrics.scala:45

...

map at MulticlassMetrics.scala:50

4 / 4

collectAsMap at MulticlassMetrics.scala:52

4 / 4

res37: Double = 0.6779446938430783

Actually not that difference suggesting that the algorithm (with the current set of parameters) isn't greatly affected by the class imbalance. This should be monitored on going as the imbalance of the data warrants this.

Its seems that spark has a confusion matrix ( what I produced in the above pivot table).Its accesses (in the book at least) via the Mllib rdd API (which is in maintenance mode now with the dataframe spark.ml api being the promoted one)

In [39]:
import org.apache.spark.mllib.evaluation.MulticlassMetrics
val predictionRDD = preds.
        select("prediction","Cover_Type").
        as[(Double,Double)].
        rdd
val multiclassMetrics = new org.apache.spark.mllib.evaluation.MulticlassMetrics(predictionRDD)

import org.apache.spark.mllib.evaluation.MulticlassMetrics

predictionRDD: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[128] at rdd at cmd38.sc:4
multiclassMetrics: MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@11200204

In [40]:
multiclassMetrics.confusionMatrix

map at MulticlassMetrics.scala:45

4 / 4

collectAsMap at MulticlassMetrics.scala:47

4 / 4

map at MulticlassMetrics.scala:55

4 / 4

collectAsMap at MulticlassMetrics.scala:57

4 / 4

res39: org.apache.spark.mllib.linalg.Matrix = 125719.0  63152.0   163.0    0.0    0.0   0.0  1475.0  
44736.0   205689.0  4207.0   114.0  64.0  0.0  189.0   
0.0       5539.0    26109.0  675.0  0.0   0.0  0.0     
0.0       15.0      1461.0   975.0  0.0   0.0  0.0     
0.0       7857.0    630.0    0.0    75.0  0.0  0.0     
0.0       5756.0    9248.0   587.0  0.0   0.0  0.0     
11780.0   171.0     52.0     0.0    0.0   0.0  6455.0  

In [41]:
predPivot.show()

show at cmd40.sc:1

4 / 4

show at cmd40.sc:1

200 / 200

show at cmd40.sc:1

200 / 200

+----------+------+------+-----+---+---+---+----+
|Cover_Type|     1|     2|    3|  4|  5|  6|   7|
+----------+------+------+-----+---+---+---+----+
|       1.0|125719| 63152|  163|  0|  0|  0|1475|
|       2.0| 44736|205689| 4207|114| 64|  0| 189|
|       3.0|     0|  5539|26109|675|  0|  0|   0|
|       4.0|     0|    15| 1461|975|  0|  0|   0|
|       5.0|     0|  7857|  630|  0| 75|  0|   0|
|       6.0|     0|  5756| 9248|587|  0|  0|   0|
|       7.0| 11780|   171|   52|  0|  0|  0|6455|
+----------+------+------+-----+---+---+---+----+



To evaluate the 70% accuracy from the default model we can benchmark it against a null model and use an inner product of the class probabilities to compare.

In [42]:
def classProbs(df: DataFrame):Array[Double] = {
    val totalCounts = df.count().asInstanceOf[Double]
        df.groupBy("Cover_Type").count()
        .orderBy("Cover_Type")
        .select("count").as[Double]
        .map(_/totalCounts)
        .collect()
}

defined function classProbs

Take inner product of test and train class probabilities as A measure of similarity

In [43]:
val trainProbs = classProbs(trainData)
val testProbs = classProbs(testData)
trainProbs.zip(testProbs).map(x=>x._1*x._2).sum

count at cmd41.sc:2

4 / 4

count at cmd41.sc:2

1 / 1

collect at cmd41.sc:6

...

collect at cmd41.sc:6

200 / 200

collect at cmd41.sc:6

8 / 8

trainProbs: Array[Double] = Array(
  4.3011328228047973E-4,
  0.36417978352876407,
  0.4874598083032891,
  0.06178911832511976,
  0.004685367354975359,
  0.01636724410171319,
  0.029803983040155375,
  0.03528458206370264
)
testProbs: Array[Double] = Array(
  3.783904645602931E-4,
  0.36688395452434597,
  0.48678213309024615,
  0.0590117129048348,
  0.005091071704993034,
  0.016012796477528766,
  0.030546430229958205,
  0.03529351060353279
)
res42_2: Double = 0.3769865498320849

The null model of randomly choosing a cover type label and using it as the prediction yields a 37% true positive rate. Makes the 70% TP rate look pretty good. 

## PIPELINES

In [44]:
import org.apache.spark.ml.Pipeline

import org.apache.spark.ml.Pipeline

In [45]:
val inputCols = trainData.columns.filter(_!="Cover_Type")
val assembler = new VectorAssembler().
                    setHandleInvalid("skip").
                    setInputCols(inputCols).
                    setOutputCol("featureVector")

inputCols: Array[String] = Array(
  "Elevation",
  "Aspect",
  "Slope",
  "Horizontal_Distance_To_Hydrology",
  "Vertical_Distance_To_Hydrology",
  "Horizontal_Distance_To_Roadways",
  "Hillshade_9am",
  "Hillshade_Noon",
  "Hillshade_3pm",
  "Horizontal_Distance_To_Fire_Points",
  "Wilderness_Area_0",
  "Wilderness_Area_1",
  "Wilderness_Area_2",
  "Wilderness_Area_3",
  "Soil_Type_0",
  "Soil_Type_1",
  "Soil_Type_2",
  "Soil_Type_3",
  "Soil_Type_4",
  "Soil_Type_5",
  "Soil_Type_6",
  "Soil_Type_7",
  "Soil_Type_8",
  "Soil_Type_9",
  "Soil_Type_10",
  "Soil_Type_11",
  "Soil_Type_12",
  "Soil_Type_13",
  "Soil_Type_14",
  "Soil_Type_15",
  "Soil_Type_16",
  "Soil_Type_17",
  "Soil_Type_18",
  "Soil_Type_19",
  "Soil_Type_20",
  "Soil_Type_21",
  "Soil_Type_22",
  "Soil_Type_23",
...
assembler: VectorAssembler = vecAssembler_f97700b8c1b8

In [46]:
val classifer = new DecisionTreeClassifier().
                setSeed(Random.nextLong()).
                setLabelCol("Cover_Type").
                setFeaturesCol("featureVector").
                setPredictionCol("prediction")

classifer: DecisionTreeClassifier = dtc_8f958317eb48

The above two transformations, recall that a transformation is a map from one dataframe to another that is part of the DAG, are chained together in a pipeline that is a convenience for defining a series of transforms on the data. We do this to streamline the hyper parameter tuning process.

In [47]:
val pipeline = new Pipeline().setStages(Array(assembler,classifier))

pipeline: Pipeline = pipeline_0a0310d29026

Use spark.ml.tuning.ParameterBuilder to set up collections of hyper parameters to try. Will obviously need an evaluation metric as well. The ParamGrid BUilder defines the series of values to trye, and the loss function

In [48]:
import org.apache.spark.ml.tuning.{ParamGridBuilder,TrainValidationSplit}

import org.apache.spark.ml.tuning.{ParamGridBuilder,TrainValidationSplit}

In [49]:
val paramGrid = new ParamGridBuilder().
                    addGrid(classifer.impurity, Seq("gini","entropy")).
                    addGrid(classifier.maxDepth, Seq(1,20)).
                    addGrid(classifier.maxBins,Seq(40,300)).
                    addGrid(classifier.minInfoGain, Seq(0.0,0.05)).
                    build()



paramGrid: Array[org.apache.spark.ml.param.ParamMap] = Array(
  {
	dtc_8f958317eb48-impurity: gini,
	dtc_d40f44a481da-maxBins: 40,
	dtc_d40f44a481da-maxDepth: 1,
	dtc_d40f44a481da-minInfoGain: 0.0
},
  {
	dtc_8f958317eb48-impurity: gini,
	dtc_d40f44a481da-maxBins: 300,
	dtc_d40f44a481da-maxDepth: 1,
	dtc_d40f44a481da-minInfoGain: 0.0
},
  {
	dtc_8f958317eb48-impurity: gini,
	dtc_d40f44a481da-maxBins: 40,
	dtc_d40f44a481da-maxDepth: 20,
	dtc_d40f44a481da-minInfoGain: 0.0
},
  {
	dtc_8f958317eb48-impurity: gini,
	dtc_d40f44a481da-maxBins: 300,
	dtc_d40f44a481da-maxDepth: 20,
	dtc_d40f44a481da-minInfoGain: 0.0
},
  {
	dtc_8f958317eb48-impurity: gini,
	dtc_d40f44a481da-maxBins: 40,
	dtc_d40f44a481da-maxDepth: 1,
	dtc_d40f44a481da-minInfoGain: 0.05
},
  {
	dtc_8f958317eb48-impurity: gini,
	dtc_d40f44a481da-maxBins: 300,
	dtc_d40f44a481da-maxDepth: 1,
	dtc_d40f44a481da-minInfoGain: 0.05
},
  {
	dtc_8f958317eb48-impurity: gini,
...

In [50]:
val multiclassEval = new MulticlassClassificationEvaluator().
setLabelCol("Cover_Type").
setPredictionCol("prediction").
setMetricName("accuracy")

multiclassEval: MulticlassClassificationEvaluator = mcEval_237bada68929

In [51]:
val validator = new TrainValidationSplit().
    setSeed(Random.nextLong()).
    setEstimator(pipeline).
    setEvaluator(multiclassEval).
    setEstimatorParamMaps(paramGrid).
    setTrainRatio(0.9)

validator: TrainValidationSplit = tvs_14fa3f270510

Now go ahead and fit the model on all the parameters set in the setEstimatorParamMaps

In [52]:
val validatorModel=validator.fit(trainData)

take at Classifier.scala:111

4 / 4

take at Classifier.scala:111

1 / 1

take at DecisionTreeMetadata.scala:112

...

mapPartitions at RandomForest.scala:538

4 / 4

collectAsMap at RandomForest.scala:557

4 / 4

validatorModel: org.apache.spark.ml.tuning.TrainValidationSplitModel = tvs_14fa3f270510

The validatorModel now contains the best model parameters for the pipeline supplied (in the setEstimator - recall that estimator is one of the spark.ml 'primatives' along with transform and is used for fitting parameters). Because a pipeline was supplied to the estimator the Model returned is wrapped in a pipeline object so we need to get the model out of the pipeline.

In [53]:
import org.apache.spark.ml.PipelineModel

import org.apache.spark.ml.PipelineModel

In [56]:
validatorModel.bestModel.asInstanceOf[PipelineModel].stages.last.extractParamMap

res55: org.apache.spark.ml.param.ParamMap = {
	dtc_d40f44a481da-cacheNodeIds: false,
	dtc_d40f44a481da-checkpointInterval: 10,
	dtc_d40f44a481da-featuresCol: featureVector,
	dtc_d40f44a481da-impurity: gini,
	dtc_d40f44a481da-labelCol: Cover_Type,
	dtc_d40f44a481da-maxBins: 300,
	dtc_d40f44a481da-maxDepth: 20,
	dtc_d40f44a481da-maxMemoryInMB: 256,
	dtc_d40f44a481da-minInfoGain: 0.0,
	dtc_d40f44a481da-minInstancesPerNode: 1,
	dtc_d40f44a481da-predictionCol: prediction,
	dtc_d40f44a481da-probabilityCol: probability,
	dtc_d40f44a481da-rawPredictionCol: rawPrediction,
	dtc_d40f44a481da-seed: -3975557625053891211
}

Its interesting that the results obtained are very different to the book. This model uses 300 bins whereas the book used only the 40. Gini index was the one that performed the best compared to the entropy that the book found. 0.0 infomration gain threshold was the same (and suggests over fitting).

To get the model out of the pipeline use the code below. note that the code val best= validatorModel.bestModel throws an error. Not clear on why but the code below returns as class with the suffix 'Model' which in spark speak is the trianed model. This should then be useable for making predictions on the test data.

In [55]:
val best = validatorModel.bestModel.asInstanceOf[PipelineModel].stages.last

best: org.apache.spark.ml.Transformer = DecisionTreeClassificationModel (uid=dtc_d40f44a481da) of depth 20 with 26913 nodes

In [58]:
val paramsAndMetrics = validatorModel.validationMetrics.zip(validatorModel.getEstimatorParamMaps).sortBy(-_._1)

paramsAndMetrics: Array[(Double, org.apache.spark.ml.param.ParamMap)] = Array(
  (
    0.9068269725434903,
    {
	dtc_8f958317eb48-impurity: gini,
	dtc_d40f44a481da-maxBins: 300,
	dtc_d40f44a481da-maxDepth: 20,
	dtc_d40f44a481da-minInfoGain: 0.0
}
  ),
  (
    0.9068269725434903,
    {
	dtc_8f958317eb48-impurity: entropy,
	dtc_d40f44a481da-maxBins: 300,
	dtc_d40f44a481da-maxDepth: 20,
	dtc_d40f44a481da-minInfoGain: 0.0
}
  ),
  (
    0.903873635272374,
    {
	dtc_8f958317eb48-impurity: gini,
	dtc_d40f44a481da-maxBins: 40,
	dtc_d40f44a481da-maxDepth: 20,
	dtc_d40f44a481da-minInfoGain: 0.0
}
  ),
  (
    0.903873635272374,
    {
	dtc_8f958317eb48-impurity: entropy,
	dtc_d40f44a481da-maxBins: 40,
	dtc_d40f44a481da-maxDepth: 20,
	dtc_d40f44a481da-minInfoGain: 0.0
}
  ),
  (
    0.6705028294876436,
...

The above .validationMetrics method shows that the top performing parameters result in increbilble close accuarcy scores so the results here are not so different to those in the book.The lesser work 4 bins might end up being a best choice. 

cmd58.sc:1: value explain is not a member of Array[(Double, org.apache.spark.ml.param.ParamMap)]
val res58 = validatorModel.validationMetrics.zip(validatorModel.getEstimatorParamMaps).sortBy(-_._1).explain
                                                                                                     ^Compilation Failed

: 

In [59]:
multiclassEval.evaluate(best.transform(testData))

: 

In [60]:
val assembledTestData = assembler.setHandleInvalid("skip").transform(testData)

assembledTestData: DataFrame = [Elevation: int, Aspect: int ... 54 more fields]

Note: because I extracted a model from the pipelineModel returned by the trainvalidationsplit training on the parameter grid then I need to assemble the data into a vector. If it was a pipelineModel ( a full pipeline returend from the training) then this would have been handled by the assembler stage in the pipeline I build.

In [61]:
multiclassEval.evaluate(best.transform(assembledTestData))

map at MulticlassMetrics.scala:45

4 / 4

collectAsMap at MulticlassMetrics.scala:47

4 / 4

countByValue at MulticlassMetrics.scala:42

4 / 4

countByValue at MulticlassMetrics.scala:42

4 / 4

res60: Double = 0.9051084843166607

The accuracy went up to 90%!

In [62]:
multiclassEval.evaluate(best.transform(assembledTrainData))

map at MulticlassMetrics.scala:45

4 / 4

collectAsMap at MulticlassMetrics.scala:47

4 / 4

countByValue at MulticlassMetrics.scala:42

4 / 4

countByValue at MulticlassMetrics.scala:42

4 / 4

res61: Double = 0.9399360863503623

The above analysis used one hot encoding of the categorical vectors. One hot encoding treats each possible value of the categorical variable as an independant feature that can take values between 0 and 1 even though these are the only two values possilble. The description in the book suggests that there is information inefficiency in doing this and that registering them as categorical variables does improve accuracy. One explaination might be the reduction in the dimension of the problem if one does not one-hot encode things. The process of registering as a categorical variable is to replace the one hot encoded data with a column whose value is a vector of length equal to the number of categories PLUS using VectorIndexer to register metadata (see pages 87 and 88 of the book for details of implementing this)